In [1]:
import os
import yaml
import json
import re
from dotenv import load_dotenv
from openai import OpenAI

## Setup

- Load OpenAI API key from .env using dotenv package
- Start client
- define functions to get embeddings from a piece of text

In [2]:
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPEN_API_TAELGAR"),
)
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

Define function to parse a markdown file into metadata and text. Probably ultimately want to insert some metadata into text and clean up markdown.

In [3]:
def parse_markdown_file(file_path):
    """
    Reads a markdown file and returns its frontmatter as a dictionary and the rest of the text as a string.

    :param file_path: Path to the markdown file.
    :return: A tuple containing a dictionary of the frontmatter and a string of the markdown text.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Check if the file starts with frontmatter (triple dashes)
    if lines and lines[0].strip() == '---':
        # Try to find the second set of triple dashes
        try:
            end_frontmatter_idx = lines[1:].index('---\n') + 1
        except ValueError:
            # Handle the case where the closing triple dashes are not found
            frontmatter = {}
            markdown_text = ''.join(lines)
        else:
            frontmatter = yaml.safe_load(''.join(lines[1:end_frontmatter_idx]))
            markdown_text = ''.join(lines[end_frontmatter_idx + 1:])
    else:
        frontmatter = {}
        markdown_text = ''.join(lines)
    return frontmatter, markdown_text

Now let's experiment with our session note summarization. 

Basic protocal will be:
- Read session note, set metadata such as date
- Set up chat API call with a system message to extract a short tagline and a description
- Run and see what happens?

In [16]:
sys_prompt = "You are a creative and careful assistant who is skilled in extracting summaries and meaningful content from text. You will receive a query that consists of possibly some optional context, followed by a potentially long text. This text will describe a narrative of one or more days, describing the events that happened in a fictional world. Your job is to summarize these narratives. You will return a JSON object that contains these things: 1. A tagline: this is a tagline of 3-8 words that could be used as a subtitle for the text; it should capture the main event of the narrative succinctly and clearly, and ALWAYS start with the words *in which* 2. A summary: this is no more than 100 words, in the form of a markdown list. each element of the list should succinctly, clearly, and accurately summarize a main event from the narrative. Choose carefully to ONLY summarize the PRIMARY OR MOST IMPORTANT parts of the narrative. 3. A short_summary; this is no more than one sentence and captures the primary gist of the narrative. 4. A location of the narrative, which can be either one or possibly two major places the events happen at or a phrase like on the road between place1 and place2, although you will prefer to choose a single location if possible. Your primary concern is summarization. Your goal is to extract the most important and relevant information from the text. You will remember that this text describes events in a fictional world. The text you receive will be formatted in markdown format, and you will ignore markdown formatting characters in your responses."
def get_session_summary(prompt, model="gpt-4-1106-preview", max_tokens=4000, system_prompt=sys_prompt):
    input_messages = []
    input_messages.append({"role": "system", "content": system_prompt})
    input_messages.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model=model,
        max_tokens=max_tokens,
        messages=input_messages,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        temperature=1,
    )
    return response

In [5]:
def split_markdown_by_sections(markdown_lines):
    """
    Splits a Markdown document (provided as a list of lines) into sections and returns a dictionary.
    The keys of the dictionary are the section names (with '#' removed), and the values are the text of each section.

    Args:
        markdown_lines (list of str): The Markdown document, split into lines.

    Returns:
        dict: A dictionary where keys are section names and values are the corresponding section text.
    """
    section_indices = [i for i, line in enumerate(markdown_lines) if re.match(r'^#+\s+.*$', line)]
    sections_dict = {}

    for i in range(len(section_indices)):
        start = section_indices[i] + 1  # Start from the line after the header
        header = markdown_lines[start - 1].lstrip('#').strip()
        end = section_indices[i + 1] if i + 1 < len(section_indices) else len(markdown_lines)
        section_content = [line for line in markdown_lines[start:end] if line.strip() != '']  # Exclude blank lines
        sections_dict[header] = '\n'.join(section_content)

    return sections_dict

In [17]:
session_note_path = "/Users/tim/Library/Mobile Documents/iCloud~md~obsidian/Documents/Taelgar/Campaigns/Dunmari Frontier/Session Notes/Session 2 (DuFr).md"
metadata, text = parse_markdown_file(session_note_path)
markdown_text = split_markdown_by_sections(text.splitlines())
narrative = ""
timeline = ""
if "Narrative" in markdown_text:
    narrative = markdown_text["Narrative"]
if "Timeline" in markdown_text:
    timeline = markdown_text["Timeline"]

if narrative and timeline:
    session_prompt = f"## Narrative\n{narrative}\n## Timeline\n{timeline}"
elif narrative:
    session_prompt = f"## Narrative\n{narrative}\n"
elif timeline:
    narrative = "\n".join([markdown_text[section] for section in markdown_text if section != "Timeline"])
    session_prompt = f"## Narrative\n{narrative}\n## Timeline\n{timeline}"
else:
    session_prompt = f"## Narrative\n{text}\n"

print(session_prompt)

## Narrative

[[Seeker]] of Highkeep, [[Wellby]] Goodbarrel, [[Kenzo]], and [[Delwath]], arriving at the [[Gomat]] Oasis past sunset, tired and weary from a long journey in the desert sun, find the oasis beset by three giant lizards, feasting on dead sheep. Deciding to attack, the party made relatively quick work of the beasts. Exhausted, and unable to determine much in the dark, the party camped for the night.

In the morning, exploring the oasis, they found signs of a battle and clear tracks of the giant lizards heading east, into the wilderness. While dissecting the lizards in search of clues to their particularly vicious and bloodthirsty behavior, an older man, wounded but alive, appeared on the horizon on horseback. Hailing him, the party learned that he was [[Akan]], one of the survivors of the giant lizard attack on [[Gomat]] the night before. His extended family had been camped at the oasis, grazing their herds, for several days, intending to rest and relax before the [[Festiva

Actually get the response from LLM

In [18]:
context = "Context: this describes events happening to a group of adventurers called the Dunmar Fellowship, occurring in the D&D world of Taelgar."
prompt = context + "\n===\n" + session_prompt
summary = get_session_summary(prompt)
print(summary)

ChatCompletion(id='chatcmpl-8jCqPW67fmrZwwKj3bYamQcqP70Xq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "tagline": "In which an oasis yields dark secrets",\n  "summary": [\n    "The Dunmar Fellowship arrives at Gomat Oasis, battling three giant lizards upon their arrival.",\n    "They encounter Akan, a survivor of a lizard attack who informs them his family was victimized while preparing for the Festival of Rebirth.",\n    "The group follows lizard tracks east, finding unusual signs of animal slaughter alongside the road.",\n    "They discover a canyon and descend, noticing converging animal tracks and a hostile, bloodthirsty environment.",\n    "At a ruined fort, they face and defeat emerging skeletons, sensing an undead presence and uncovering traces of recent disturbance."\n  ],\n  "short_summary": "The Dunmar Fellowship uncovers a malign presence and battles undead at a beleaguered Gomat Oasis and an ancient for

Format of a session note should be:

# {campaign} Session {sessionNumber}
*{tagline}*

**In the real world**: {realWorldDate}
**In Taelgar**: {DR} - {DR_end} %% unless DR == DR_end, in which case just {DR}
**{location}**

## Summary
{summary}

## Session Events

(can include: timeline, stories/intro, mirror uses, level ups, new treasure, anything that is relevant but not narrative)


## Narrative
{narrative}


In [19]:
clean_resp = summary.choices[0].message.content.replace("```", "").replace("json", "").strip()
resp_data = json.loads(clean_resp)
print(resp_data)
tagline = resp_data["tagline"]
print("*" + tagline[0].lower() + tagline[1:] + "*")
print("\n## Summary\n    - ", end="")
print("\n    - ".join(resp_data["summary"]))
print("\n## Short Summary\n", end="")
print(resp_data["short_summary"])
print("\n## Location", end="")
print("\n    - " + resp_data["location"])


{'tagline': 'In which an oasis yields dark secrets', 'summary': ['The Dunmar Fellowship arrives at Gomat Oasis, battling three giant lizards upon their arrival.', 'They encounter Akan, a survivor of a lizard attack who informs them his family was victimized while preparing for the Festival of Rebirth.', 'The group follows lizard tracks east, finding unusual signs of animal slaughter alongside the road.', 'They discover a canyon and descend, noticing converging animal tracks and a hostile, bloodthirsty environment.', 'At a ruined fort, they face and defeat emerging skeletons, sensing an undead presence and uncovering traces of recent disturbance.'], 'short_summary': 'The Dunmar Fellowship uncovers a malign presence and battles undead at a beleaguered Gomat Oasis and an ancient fort.', 'location': 'Gomat Oasis and the ruined fort eastward'}
*in which an oasis yields dark secrets*

## Summary
    - The Dunmar Fellowship arrives at Gomat Oasis, battling three giant lizards upon their arriv